In [9]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2

def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [10]:
gt_folder = './GEE_Masks/GEE_resized/'
pred_folder = './GEE_Output/'

# List all files in the prediction folder
# pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# Initialize metrics



In [11]:
# Lists to store metrics
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

for i in range(0, 1263):  # Adjust the range based on the number of files
    gt_filename = f'NDWI_Mask_{i}_resized.tif'
    pred_filename = f'dense_Predicted_Image_{i}.tif'
    
    gt_path = os.path.join(gt_folder, gt_filename)
    pred_path = os.path.join(pred_folder, pred_filename)
    
    # Load the prediction and ground truth images
    pred_image = load_image(pred_path)
    gt_image = load_image(gt_path)
    
    # Flatten the images
    pred_flat = pred_image.flatten()
    gt_flat = gt_image.flatten()
    
    # Calculate metrics
    dice = dice_coefficient(gt_flat, pred_flat)
    iou = jaccard_score(gt_flat, pred_flat, average='macro')
    accuracy = pixel_accuracy(gt_flat, pred_flat)
    precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
    recall = recall_score(gt_flat, pred_flat, average='macro')
    f1 = f1_score(gt_flat, pred_flat, average='macro')
    spec = specificity(gt_flat, pred_flat)
    
    # Store metrics
    dice_scores.append(dice)
    iou_scores.append(iou)
    pixel_accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    specificities.append(spec)
    
    # Print metrics for the current image
    print(f"Metrics for {gt_filename} and {pred_filename}:")
    print(f"  Dice Coefficient: {dice:.4f}")
    print(f"  IoU: {iou:.4f}")
    print(f"  Pixel Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  Specificity: {spec:.4f}")
    print()

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Metrics for NDWI_Mask_0_resized.tif and dense_Predicted_Image_0.tif:
  Dice Coefficient: 0.8730
  IoU: 0.8507
  Pixel Accuracy: 0.9415
  Precision: 0.9326
  Recall: 0.9046
  F1 Score: 0.9175
  Specificity: 0.9759

Metrics for NDWI_Mask_1_resized.tif and dense_Predicted_Image_1.tif:
  Dice Coefficient: 0.9904
  IoU: 0.9652
  Pixel Accuracy: 0.9859
  Precision: 0.9826
  Recall: 0.9818
  F1 Score: 0.9822
  Specificity: 0.9729

Metrics for NDWI_Mask_2_resized.tif and dense_Predicted_Image_2.tif:
  Dice Coefficient: 0.8231
  IoU: 0.8242
  Pixel Accuracy: 0.9543
  Precision: 0.9472
  Recall: 0.8626
  F1 Score: 0.8985
  Specificity: 0.9918

Metrics for NDWI_Mask_3_resized.tif and dense_Predicted_Image_3.tif:
  Dice Coefficient: 0.5455
  IoU: 0.6618
  Pixel Accuracy: 0.9501
  Precision: 0.9590
  Recall: 0.6893
  F1 Score: 0.7596
  Specificity: 0.9989

Metrics for NDWI_Mask_4_resized.tif and dense_Predicted_Image_4.tif:
  Dice Coefficient: 0.6462
  IoU: 0.6225
  Pixel Accuracy: 0.8084
  Precisi

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_56_resized.tif and dense_Predicted_Image_56.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4993
  Pixel Accuracy: 0.9986
  Precision: 0.5000
  Recall: 0.4993
  F1 Score: 0.4997
  Specificity: 0.9986

Metrics for NDWI_Mask_57_resized.tif and dense_Predicted_Image_57.tif:
  Dice Coefficient: 0.9849
  IoU: 0.9768
  Pixel Accuracy: 0.9892
  Precision: 0.9888
  Recall: 0.9877
  F1 Score: 0.9882
  Specificity: 0.9930

Metrics for NDWI_Mask_58_resized.tif and dense_Predicted_Image_58.tif:
  Dice Coefficient: 0.8809
  IoU: 0.8733
  Pixel Accuracy: 0.9648
  Precision: 0.9044
  Recall: 0.9617
  F1 Score: 0.9301
  Specificity: 0.9659

Metrics for NDWI_Mask_59_resized.tif and dense_Predicted_Image_59.tif:
  Dice Coefficient: 0.8040
  IoU: 0.8357
  Pixel Accuracy: 0.9992
  Precision: 0.9772
  Recall: 0.8470
  F1 Score: 0.9018
  Specificity: 0.9999



c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_60_resized.tif and dense_Predicted_Image_60.tif:
  Dice Coefficient: 0.0000
  IoU: 0.3245
  Pixel Accuracy: 0.6489
  Precision: 0.5000
  Recall: 0.3245
  F1 Score: 0.3935
  Specificity: 0.6489

Metrics for NDWI_Mask_61_resized.tif and dense_Predicted_Image_61.tif:
  Dice Coefficient: 0.9697
  IoU: 0.9562
  Pixel Accuracy: 0.9803
  Precision: 0.9775
  Recall: 0.9776
  F1 Score: 0.9776
  Specificity: 0.9853

Metrics for NDWI_Mask_62_resized.tif and dense_Predicted_Image_62.tif:
  Dice Coefficient: 0.9849
  IoU: 0.9839
  Pixel Accuracy: 0.9978
  Precision: 0.9929
  Recall: 0.9907
  F1 Score: 0.9918
  Specificity: 0.9990

Metrics for NDWI_Mask_63_resized.tif and dense_Predicted_Image_63.tif:
  Dice Coefficient: 0.9798
  IoU: 0.9714
  Pixel Accuracy: 0.9877
  Precision: 0.9843
  Recall: 0.9867
  F1 Score: 0.9855
  Specificity: 0.9892

Metrics for NDWI_Mask_64_resized.tif and dense_Predicted_Image_64.tif:
  Dice Coefficient: 0.0002
  IoU: 0.4441
  Pixel Accuracy: 0.8881

...

Metrics for NDWI_Mask_1229_resized.tif and dense_Predicted_Image_1229.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4253
  Pixel Accuracy: 0.8506
  Precision: 0.5000
  Recall: 0.4253
  F1 Score: 0.4596
  Specificity: 0.8506

Metrics for NDWI_Mask_1230_resized.tif and dense_Predicted_Image_1230.tif:
  Dice Coefficient: 0.2913
  IoU: 0.5823
  Pixel Accuracy: 0.9941
  Precision: 0.8728
  Recall: 0.5901
  F1 Score: 0.6442
  Specificity: 0.9996



c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1231_resized.tif and dense_Predicted_Image_1231.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4380
  Pixel Accuracy: 0.8760
  Precision: 0.5000
  Recall: 0.4380
  F1 Score: 0.4669
  Specificity: 0.8760

Metrics for NDWI_Mask_1232_resized.tif and dense_Predicted_Image_1232.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4411
  Pixel Accuracy: 0.8821
  Precision: 0.5000
  Recall: 0.4411
  F1 Score: 0.4687
  Specificity: 0.8821

Metrics for NDWI_Mask_1233_resized.tif and dense_Predicted_Image_1233.tif:
  Dice Coefficient: 0.5349
  IoU: 0.6783
  Pixel Accuracy: 0.9916
  Precision: 0.9134
  Recall: 0.6963
  F1 Score: 0.7653
  Specificity: 0.9990

Metrics for NDWI_Mask_1234_resized.tif and dense_Predicted_Image_1234.tif:
  Dice Coefficient: 0.9541
  IoU: 0.9438
  Pixel Accuracy: 0.9804
  Precision: 0.9780
  Recall: 0.9640
  F1 Score: 0.9708
  Specificity: 0.9930

Metrics for NDWI_Mask_1235_resized.tif and dense_Predicted_Image_1235.tif:
  Dice Coefficient: 0.9710
  IoU: 0.9677
  Pi

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1243_resized.tif and dense_Predicted_Image_1243.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4091
  Pixel Accuracy: 0.8181
  Precision: 0.5000
  Recall: 0.4091
  F1 Score: 0.4500
  Specificity: 0.8182

Metrics for NDWI_Mask_1244_resized.tif and dense_Predicted_Image_1244.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4898
  Pixel Accuracy: 0.9796
  Precision: 0.5000
  Recall: 0.4898
  F1 Score: 0.4948
  Specificity: 0.9796

Metrics for NDWI_Mask_1245_resized.tif and dense_Predicted_Image_1245.tif:
  Dice Coefficient: 0.4281
  IoU: 0.5512
  Pixel Accuracy: 0.8402
  Precision: 0.6362
  Recall: 0.9130
  F1 Score: 0.6676
  Specificity: 0.8303

Metrics for NDWI_Mask_1246_resized.tif and dense_Predicted_Image_1246.tif:
  Dice Coefficient: 0.9740
  IoU: 0.9681
  Pixel Accuracy: 0.9894
  Precision: 0.9883
  Recall: 0.9793
  F1 Score: 0.9837
  Specificity: 0.9965

Metrics for NDWI_Mask_1247_resized.tif and dense_Predicted_Image_1247.tif:
  Dice Coefficient: 0.9924
  IoU: 0.9923
  Pi

In [12]:
# Print average metrics
print("Average Metrics:")
print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
print(f"  Average IoU: {np.mean(iou_scores):.4f}")
print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
print(f"  Average Precision: {np.mean(precisions):.4f}")
print(f"  Average Recall: {np.mean(recalls):.4f}")
print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
print(f"  Average Specificity: {np.mean(specificities):.4f}")

Average Metrics:
  Average Dice Coefficient: 0.7131
  Average IoU: 0.7875
  Average Pixel Accuracy: 0.9406
  Average Precision: 0.8632
  Average Recall: 0.8495
  Average F1 Score: 0.8357
  Average Specificity: 0.9595
